<a href="https://colab.research.google.com/github/eviatar-ben/Audio-And-Speech-Processing/blob/main/Audio_final_project_collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install wandb -qU
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.2 MB/s eta 0:00:00


In [5]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
!gdown "1MiPqJDm6gX_ayXZJ2LHeUbG0UNZfNagF"

Downloading...
From: https://drive.google.com/uc?id=1MiPqJDm6gX_ayXZJ2LHeUbG0UNZfNagF
To: /content/an4.zip
100% 64.6M/64.6M [00:01<00:00, 49.1MB/s]


In [7]:
!unzip an4.zip

Archive:  an4.zip
   creating: an4/
  inflating: __MACOSX/._an4          
  inflating: an4/.DS_Store           
  inflating: __MACOSX/an4/._.DS_Store  
   creating: an4/test/
  inflating: __MACOSX/an4/._test     
   creating: an4/train/
  inflating: __MACOSX/an4/._train    
   creating: an4/val/
  inflating: __MACOSX/an4/._val      
  inflating: an4/test/.DS_Store      
  inflating: __MACOSX/an4/test/._.DS_Store  
   creating: an4/test/an4/
  inflating: __MACOSX/an4/test/._an4  
  inflating: an4/train/.DS_Store     
  inflating: __MACOSX/an4/train/._.DS_Store  
   creating: an4/train/an4/
  inflating: __MACOSX/an4/train/._an4  
   creating: an4/val/an4/
  inflating: __MACOSX/an4/val/._an4  
   creating: an4/test/an4/wav/
  inflating: __MACOSX/an4/test/an4/._wav  
  inflating: an4/test/an4/.DS_Store  
  inflating: __MACOSX/an4/test/an4/._.DS_Store  
   creating: an4/test/an4/txt/
  inflating: __MACOSX/an4/test/an4/._txt  
   creating: an4/train/an4/wav/
  inflating: __MACOSX/an4/train/a

In [92]:
import os
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
import numpy as np
import wandb
from jiwer import wer
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

In [93]:
class TextTransform:
    """Maps characters to integers and vice versa"""

    def __init__(self):
        char_map_str = """
        ' 0
        <SPACE> 1
        a 2
        b 3
        c 4
        d 5
        e 6
        f 7
        g 8
        h 9
        i 10
        j 11
        k 12
        l 13
        m 14
        n 15
        o 16
        p 17
        q 18
        r 19
        s 20
        t 21
        u 22
        v 23
        w 24
        x 25
        y 26
        z 27
        """
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[1] = ' '

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['<SPACE>']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('<SPACE>', ' ')


text_transform = TextTransform()


def greedy_decoder(output, labels, label_lengths, blank_label=28, collapse_repeated=True):
    """
    Greedy Decoder. Decodes the output of the network by picking the label with the highest probability at each time step.
    """
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j - 1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets

In [94]:
WB = False

learning_rate = 5e-4
batch_size = 10
epochs = 20

hparams = {
    "rnn_dim": 128,
    "n_class": 29,
    "n_feats": 13,
    "stride": 2,
    "dropout": 0.1,
    "learning_rate": learning_rate,
    "batch_size": batch_size,
    "epochs": epochs
}

In [95]:
np.random.seed(42)

class BatchIterator:
    def __init__(self, x, y, input_lengths, label_lengths, batch_size, shuffle=True):
        self.x = x
        self.y = y
        self.input_lengths = input_lengths
        self.label_lengths = label_lengths
        self.batch_size = batch_size
        self.num_samples = len(x)
        self.num_batches = (self.num_samples + batch_size - 1) // batch_size
        self.shuffle = shuffle
        self.current_batch = 0

    def __iter__(self):
        if self.shuffle:
            indices = np.random.permutation(self.num_samples)
            self.x = self.x[indices]
            self.y = self.y[indices]
            self.input_lengths = np.asarray(self.input_lengths)[indices]
            self.label_lengths = np.asarray(self.label_lengths)[indices]
        self.current_batch = 0
        return self

    def __len__(self):
        return self.num_batches

    def __next__(self):
        if self.current_batch < self.num_batches:
            start_idx = self.current_batch * self.batch_size
            end_idx = min((self.current_batch + 1) * self.batch_size, self.num_samples)

            batch_x = self.x[start_idx:end_idx]
            batch_y = self.y[start_idx:end_idx]
            batch_input_lengths = self.input_lengths[start_idx:end_idx]
            batch_label_lengths = self.label_lengths[start_idx:end_idx]

            self.current_batch += 1

            return batch_x, batch_y, batch_input_lengths, batch_label_lengths
        else:
            raise StopIteration


def load_wavs_data(load_again=False, save=False,
                   path=r"/content/an4"):
    text_transform = TextTransform()

    if load_again:
        all_spectrogram = torch.load("data/all_spectrogram.pt")
        all_labels = torch.load("data/all_labels.pt")
        all_input_lengths = torch.load("data/all_input_lengths.pt")
        all_label_lengths = torch.load("data/all_label_lengths.pt")
        return all_spectrogram, all_labels, all_input_lengths, all_label_lengths

    valid_file = ["test", "train", "val"]
    all_spectrogram = {"test": [], "train": [], "val": []}
    all_labels = {"test": [], "train": [], "val": []}
    all_input_lengths = {"test": [], "train": [], "val": []}
    all_label_lengths = {"test": [], "train": [], "val": []}
    for dir in os.listdir(path):
        if dir in valid_file:
            spectrogram = []
            labels = []
            input_lengths = []
            label_lengths = []
            for root2, dirs2, files2 in os.walk(os.path.join(path, dir)):
                for file in files2:
                    if file.endswith(".txt"):
                        # change suffix to wav
                        wav = file.replace(".txt", ".wav")
                        # print(os.path.join(root2, file))
                        # change last dir to wav instead of txt
                        root2_wav = root2.replace("txt", "wav")

                        # load wav:
                        waveform, sample_rate = torchaudio.load(os.path.join(root2_wav, wav))
                        # mfcc:
                        mfcc = torchaudio.transforms.MFCC(sample_rate=sample_rate, n_mfcc=13)(waveform)
                        mfcc = mfcc.squeeze(0)
                        mfcc = mfcc.transpose(0, 1)
                        # add mfcc to y:
                        spectrogram.append(mfcc)

                        # load txt:
                        with open(os.path.join(root2, file), 'r') as f:
                            text = f.read()
                            int_text = text_transform.text_to_int(text.lower())
                        # add text to labels:
                        label = torch.Tensor(int_text)
                        labels.append(label)
                        input_lengths.append(mfcc.shape[0] // 2)  # todo why divide by 2?
                        label_lengths.append(len(label))
                    # print(file)

            # todo maybe the padding should be done for all the data together (train test and val)
            spectrogram = nn.utils.rnn.pad_sequence(spectrogram, batch_first=True).unsqueeze(1).transpose(2, 3)
            labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

            all_spectrogram[dir] = spectrogram
            all_labels[dir] = labels
            all_input_lengths[dir] = input_lengths
            all_label_lengths[dir] = label_lengths

    if save:
        torch.save(all_spectrogram, "data/all_spectrogram.pt")
        torch.save(all_labels, "data/all_labels.pt")
        torch.save(all_input_lengths, "data/all_input_lengths.pt")
        torch.save(all_label_lengths, "data/all_label_lengths.pt")
    return all_spectrogram, all_labels, all_input_lengths, all_label_lengths


def get_batch_iterator(data_type, batch_size=hparams["batch_size"]):
    if data_type not in ["test", "train", "val"]:
        raise ValueError("data_type must be one of [test, train, val]")
    all_spectrogram, all_labels, all_input_lengths, all_label_lengths = load_wavs_data(load_again=False, save=False)
    return BatchIterator(all_spectrogram[data_type], all_labels[data_type],
                         all_input_lengths[data_type], all_label_lengths[data_type], batch_size)

In [100]:
class ResCNN(nn.Module):
    def __init__(self, n_cnn_layers, n_class, n_feats, stride=2, dropout=0.1):
        super(ResCNN, self).__init__()
        n_feats = n_feats // 2 + 1
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride,
                             padding=3 // 2)  # cnn for extracting heirachal features
        self.rescnn_layers = nn.Sequential(*[
            ResBlock(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats)
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats * 32, n_class)

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])
        x = x.transpose(1, 2)
        x = self.fully_connected(x)
        return x


class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""

    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous()  # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous()  # (batch, channel, feature, time)


class ResBlock(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """

    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResBlock, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel // 2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel // 2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x  # (batch, channel, feature, time)


class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.relu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


class ResCNNTransformer(nn.Module):
    def __init__(self, n_cnn_layers, n_class, n_feats, dropout=0.1):
        super(ResCNNTransformer, self).__init__()
        n_feats = n_feats // 2 + 1
        self.cnn = nn.Conv2d(1, 32, 3, stride=2, padding=3 // 2)  # cnn for extracting heirachal features
        self.rescnn_layers = nn.Sequential(*[
            ResBlock(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats)
            for _ in range(n_cnn_layers)
        ])
        self.transformer = nn.Transformer(nhead=4, num_encoder_layers=6, num_decoder_layers=6, dropout=dropout, d_model=n_feats * 32)
        self.fully_connected = nn.Linear(n_feats * 32, n_class)

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2)  # (batch, time, feature)
        x = self.transformer(x, x)
        x = self.fully_connected(x)
        return x


class ResCNNMultiTransformer(nn.Module):
    """
    the same as ResCNNTransformer but with multiple parallel transformers
    """
    def __init__(self, n_cnn_layers, n_class, n_feats, dropout=0.1):
        super(ResCNNMultiTransformer, self).__init__()
        n_feats = n_feats // 2 + 1
        self.cnn = nn.Conv2d(1, 32, 3, stride=2, padding=3 // 2)  # cnn for extracting heirachal features
        self.rescnn_layers = nn.Sequential(*[
            ResBlock(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats)
            for _ in range(n_cnn_layers)
        ])
        self.transformer1 = nn.Transformer(nhead=2, num_encoder_layers=4, num_decoder_layers=4, dropout=dropout, d_model=n_feats * 32)
        self.transformer2 = nn.Transformer(nhead=2, num_encoder_layers=4, num_decoder_layers=4, dropout=dropout, d_model=n_feats * 32)
        self.transformer3 = nn.Transformer(nhead=2, num_encoder_layers=4, num_decoder_layers=4, dropout=dropout, d_model=n_feats * 32)
        self.fully_connected = nn.Linear(n_feats * 32 * 3, n_class)

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2)  # (batch, time, feature)
        x1 = self.transformer1(x, x)
        x2 = self.transformer2(x, x)
        x3 = self.transformer3(x, x)
        x = torch.cat((x1, x2, x3), dim=2)
        x = self.fully_connected(x)
        return x


class RNN(nn.Module):
    def __init__(self, n_cnn_layers, n_class, n_feats, dropout=0.1, hidden_size=128, num_layers=2):
        super(RNN, self).__init__()
        n_feats = n_feats // 2 + 1
        self.cnn = nn.Conv2d(1, 32, 3, stride=2, padding=3 // 2)
        self.rescnn_layers = nn.Sequential(*[
            ResBlock(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats)
            for _ in range(n_cnn_layers)
        ])
        self.rnn = nn.RNN(input_size=n_feats * 32, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.fully_connected = nn.Linear(hidden_size, n_class)

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2)  # (batch, time, feature)
        x, _ = self.rnn(x)  # Use only the output, disregard hidden states
        x = self.fully_connected(x)
        return x

class DeepSpeechModel(nn.Module):

    def __init__(self, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(DeepSpeechModel, self).__init__()
        n_feats = n_feats // 2 + 1  # todo even vrsos odd
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3 // 2)  # cnn for extracting heirachal features

        self.rescnn1 = ResBlock(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats)
        self.rescnn2 = ResBlock(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats)
        self.rescnn3 = ResBlock(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats)
        self.fully_connected = nn.Linear(n_feats * 32, rnn_dim)

        self.rnn1 = BidirectionalGRU(rnn_dim=rnn_dim,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=True)
        self.rnn2 = BidirectionalGRU(rnn_dim=rnn_dim * 2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=False)
        self.relu = nn.ReLU()
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim * 2, rnn_dim),  # birnn returns rnn_dim*2
            self.relu,
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn1(x)
        x = self.rescnn2(x)
        x = self.rescnn3(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2)  # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.rnn1(x)
        x = self.rnn2(x)
        x = self.classifier(x)
        return x




In [101]:
from tqdm import tqdm

def train(model, device, batch_iterator, criterion, optimizer, scheduler, epoch):
    model.train()
    data_len = len(batch_iterator)
    for batch_idx, _data in enumerate(batch_iterator):
        spectrograms, labels, input_lengths, label_lengths = _data
        spectrograms, labels = spectrograms.to(device), labels.to(device)

        optimizer.zero_grad()

        output = model(spectrograms)  # (batch, time, n_class)
        output = F.log_softmax(output, dim=2)  # using log_softmax instead of softmax for numerical stability
        output = output.transpose(0, 1)  # (time, batch, n_class)

        loss = criterion(output, labels, torch.from_numpy(input_lengths), torch.from_numpy(label_lengths))
        loss.backward()

        optimizer.step()
        scheduler.step()


        # log to wandb once every epoch:
        if batch_idx == data_len - 1 and WB:
            wandb.log({"train_loss": loss.item()}, step=epoch)
            decoded_preds, decoded_targets = greedy_decoder(output.transpose(0, 1), labels,
                                                                  label_lengths)
            wer_sum = 0
            for j in range(len(decoded_preds)):
                wer_sum += wer(decoded_targets[j], decoded_preds[j])

            wandb.log({"train_wer": wer_sum / len(decoded_preds)}, step=epoch)


def validation(model, device, val_loader, criterion, epoch):
    model.eval()
    val_loss = 0
    val_wer = []
    with torch.no_grad():
        for i, _data in enumerate(val_loader):
            spectrograms, labels, input_lengths, label_lengths = _data
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1)  # (time, batch, n_class)

            loss = criterion(output, labels, torch.from_numpy(input_lengths), torch.from_numpy(label_lengths))
            val_loss += loss.item() / len(val_loader)

            decoded_preds, decoded_targets = greedy_decoder(output.transpose(0, 1), labels, label_lengths)
            for j in range(len(decoded_preds)):
                val_wer.append(wer(decoded_targets[j], decoded_preds[j]))

    avg_wer = sum(val_wer) / len(val_wer)
    # experiment.log_metric('val_loss', val_loss, step=iter_meter.get())
    # experiment.log_metric('wer', avg_wer, step=iter_meter.get())



    # print a sample of the val data and decoded predictions against the true labels
    if epoch % 20 == 0:
        print(
        'val set: Average loss: {:.4f}, Average WER: {:.4f}\n'.format(val_loss, avg_wer))
        print('Ground Truth -> Decoded Prediction')
        for i in range(10):
            print('{} -> {}'.format(decoded_targets[i], decoded_preds[i]))

    if WB:
        wandb.log({"val_loss": val_loss})
        wandb.log({"val_wer": avg_wer})


def train_and_validation(hparams, batch_iterators):
    train_loader = batch_iterators[0]
    val_loader = batch_iterators[1]
    epochs = hparams['epochs']
    model = DeepSpeechModel(hparams['rnn_dim'], hparams['n_class'],
                                   hparams['n_feats'], hparams['stride'],
                                   hparams['dropout']
                                   ).to(device)

    optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
    criterion = nn.CTCLoss(blank=28).to(device)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'],
                                              steps_per_epoch=len(train_loader),  # todo: check if this is correct
                                              epochs=hparams['epochs'],
                                              anneal_strategy='linear')

    for epoch in tqdm(range(1, epochs + 1)):
        train(model, device, train_loader, criterion, optimizer, scheduler, epoch)
        validation(model, device, val_loader, criterion, epoch)

In [102]:
DESCRIPTION = 'initial work'
RUN = 'Complex Model'


def init_w_and_b():
    epochs = hparams['epochs']
    learning_rate = hparams['learning_rate']

    if WB:
        wandb.init(
            # Set the project where this run will be logged
            group="Complex Model initial work",
            project="ASR",
            # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
            name=f"{DESCRIPTION}{RUN}_{hparams}_epochs",
            notes='checking if log is work properly',
            # Track hyperparameters and run metadata
            config={
                "learning_rate": learning_rate,
                "architecture": "assembly",
                "dataset": "AN4",
                "epochs": epochs,

            })


def main():
    if WB:
        wandb.login()
        init_w_and_b()

    train_batch_iterator = get_batch_iterator("train", hparams["batch_size"])
    test_batch_iterator = get_batch_iterator("test", hparams["batch_size"])
    val_batch_iterator = get_batch_iterator("val", hparams["batch_size"])
    all_iterators = [train_batch_iterator, test_batch_iterator, val_batch_iterator]
    train_and_validation(hparams, all_iterators)

    if WB:
        wandb.finish()

In [103]:
main()

/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(
100%|██████████| 20/20 [01:26<00:00,  4.31s/it]

val set: Average loss: 1.0996, Average WER: 0.7194

Ground Truth -> Decoded Prediction
a b v j ninety four ->    h iny four
s y f c d nine six oh one seven -> s  c n ne six o on seven
one five two two seven -> oe five twotwo seven
go -> r
enter nine one six nine -> ntr ne one six ne
enter seven -> en tr seven
seven one six two four four six seven one four -> seven on six two fur four six seveen on four
l k k n thirty eight ->  k t  ter e iht
p i t t s b u r g h -> p  t ts b r h
no -> o
